In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../input/train.csv')
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
def transform(frame):
    df = frame.copy()
    df[['Group','GroupMember']] = df['PassengerId'].str.split('_', expand=True)
    df[['Deck','Room','Side']] = df['Cabin'].str.split('/', expand=True)
    df = df.drop(columns='Cabin')

    return df

df = transform(df)

## Detecting column types

**General Plan**
- Use `dropna().convert_dtypes()` to determine most likely datatype
- For string columns
    - Is this numerical data encoded as a string?
    - Text data vs Categorical?
- For number columns

In [5]:
text_cols = []
num_cols = []

for col in df:
    
    print(f'{col} \t {df[col].dropna().convert_dtypes().dtypes}')

PassengerId 	 string
HomePlanet 	 string
CryoSleep 	 boolean
Destination 	 string
Age 	 Int64
VIP 	 boolean
RoomService 	 Int64
FoodCourt 	 Int64
ShoppingMall 	 Int64
Spa 	 Int64
VRDeck 	 Int64
Name 	 string
Transported 	 boolean
Group 	 string
GroupMember 	 string
Deck 	 string
Room 	 string
Side 	 string


In [6]:
df.dropna()\
    .convert_dtypes()\
    .select_dtypes(exclude=['object','category','bool','string'])\
    .columns.tolist()

['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

![https://i.stack.imgur.com/g1phN.png](https://i.stack.imgur.com/g1phN.png)

Okay, so to do what I want to do I think I need to collect some metadata about the column.  I'll start with `Age`, which I am considering to be a continuous variable.

In [7]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupMember,Deck,Room,Side
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,01,B,0,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,01,F,0,S
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,01,A,0,S
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,02,A,0,S
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,01,F,1,S


In [8]:
column = df['Room']

metadata = {
    'orig_dtype': column.dtypes,
    'guessed_dtype': column.dropna().convert_dtypes().dtypes,
    'rows' : len(column),
    'nulls': column.isna().sum(),
    'unique_values': len(column.unique()),
    'pct_unique': len(column.unique()) /  len(column)
}


# Assume everything is categorical unless we find reason to suspect otherwise.
categorical = True
numeric_score = 0

if np.issubdtype(metadata['orig_dtype'], float):
    numeric_score += .8
if np.issubdtype(metadata['orig_dtype'], int):
    numeric_score += .7
if metadata['pct_unique'] < 0.30:
    numeric_score += .2

numeric_score, metadata


(0.2,
 {'orig_dtype': dtype('O'),
  'guessed_dtype': string[python],
  'rows': 8693,
  'nulls': 199,
  'unique_values': 1818,
  'pct_unique': 0.20913378580467043})

In [9]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport



In [10]:
profile = ProfileReport(df, title="Pandas Profiling Report", minimal=True)

In [11]:
profile.to_file("minimal.json")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Python\3.10.8\lib\site-packages\multimethod\__init__.py:315: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  return func(*args, **kwargs)


Render JSON:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
import json
with open('minimal.json') as f:
    data = json.load(f)

In [14]:
data.keys()

dict_keys(['analysis', 'table', 'variables', 'scatter', 'correlations', 'missing', 'alerts', 'package', 'sample', 'duplicates'])

In [22]:
data['package']

{'pandas_profiling_version': '3.4.0',
 'pandas_profiling_config': '{"title": "Pandas Profiling Report", "dataset": {"description": "", "creator": "", "author": "", "copyright_holder": "", "copyright_year": "", "url": ""}, "variables": {"descriptions": {}}, "infer_dtypes": false, "show_variable_description": true, "pool_size": 0, "progress_bar": true, "vars": {"num": {"quantiles": [0.05, 0.25, 0.5, 0.75, 0.95], "skewness_threshold": 20, "low_categorical_threshold": 5, "chi_squared_threshold": 0.0}, "cat": {"length": false, "characters": false, "words": false, "cardinality_threshold": 50, "n_obs": 5, "chi_squared_threshold": 0.0, "coerce_str_to_date": false, "redact": false, "histogram_largest": 10, "stop_words": []}, "image": {"active": false, "exif": false, "hash": false}, "bool": {"n_obs": 3, "mappings": {"t": true, "f": false, "True": true, "False": false, "y": true, "n": false}}, "path": {"active": false}, "file": {"active": false}, "url": {"active": false}, "timeseries": {"active":

In [20]:
data['alerts']

['[HIGH_CARDINALITY] alert on column PassengerId',
 '[HIGH_CARDINALITY] alert on column Name',
 '[HIGH_CARDINALITY] alert on column Group',
 '[HIGH_CARDINALITY] alert on column Room',
 '[MISSING] alert on column HomePlanet',
 '[MISSING] alert on column CryoSleep',
 '[MISSING] alert on column Destination',
 '[MISSING] alert on column Age',
 '[MISSING] alert on column VIP',
 '[MISSING] alert on column RoomService',
 '[MISSING] alert on column FoodCourt',
 '[MISSING] alert on column ShoppingMall',
 '[MISSING] alert on column Spa',
 '[MISSING] alert on column VRDeck',
 '[MISSING] alert on column Name',
 '[MISSING] alert on column Deck',
 '[MISSING] alert on column Room',
 '[MISSING] alert on column Side',
 '[UNIQUE] alert on column PassengerId',
 '[ZEROS] alert on column Age',
 '[ZEROS] alert on column RoomService',
 '[ZEROS] alert on column FoodCourt',
 '[ZEROS] alert on column ShoppingMall',
 '[ZEROS] alert on column Spa',
 '[ZEROS] alert on column VRDeck']

In [21]:
data['variables']['Group']

{'n_distinct': 6217,
 'p_distinct': 0.7151731278039802,
 'is_unique': False,
 'n_unique': 4805,
 'p_unique': 0.5527435867939722,
 'type': 'Categorical',
 'hashable': True,
 'value_counts_without_nan': {'4498': 8,
  '8168': 8,
  '8728': 8,
  '8796': 8,
  '8956': 8,
  '4256': 8,
  '0984': 8,
  '9081': 8,
  '8988': 8,
  '5756': 8,
  '5885': 8,
  '4005': 8,
  '5133': 8,
  '4176': 7,
  '1211': 7,
  '0714': 7,
  '6363': 7,
  '6787': 7,
  '8770': 7,
  '4172': 7,
  '0221': 7,
  '5927': 7,
  '6443': 7,
  '9157': 7,
  '5814': 7,
  '6334': 7,
  '0378': 7,
  '1709': 7,
  '0220': 7,
  '0571': 7,
  '1500': 7,
  '6020': 7,
  '8383': 7,
  '4001': 7,
  '4625': 7,
  '4661': 7,
  '2234': 7,
  '1976': 7,
  '3868': 7,
  '3862': 7,
  '1148': 7,
  '3850': 7,
  '2384': 7,
  '6030': 7,
  '2347': 7,
  '6302': 7,
  '7872': 6,
  '6139': 6,
  '6921': 6,
  '1865': 6,
  '5090': 6,
  '9074': 6,
  '8157': 6,
  '6137': 6,
  '3353': 6,
  '2055': 6,
  '6816': 6,
  '3995': 6,
  '6971': 6,
  '0992': 6,
  '8202': 6,
  '1829